## Importation des modules

In [55]:
# -*- coding: utf-8 -*-
import numpy as np
from scipy.optimize import fsolve

## Résolution numérique

### Définition des paramètres géométriques

In [56]:
L_x, L_y = 10, 5 # Longueur et largeur de notre domaine d'étude
N_x, N_y = 100, 50 # Nombre de mailles selon x et selon y

Attention à ne pas confondre **nombre de mailles** et **nombre de points**.

Typiquement, on a choisi de représenter l'intervalle continu $[0, L_x]$ par l'intervalle discret de points $\llbracket 0, N_x \rrbracket$. Ainsi, il y a bien $N_x$ **mailles**, de la forme $[x_i, x_{i+1}]$, mais **Nx + 1 points**.

De même, selon y, il y a $N_y$ **mailles** et donc $N_y+1$ **points**.

In [57]:
dx = L_x/N_x # Taille d'une maille selon x
dy = L_y/N_y # Taille d'une maille selon y

In [58]:
dt = 1 # Choix du pas de temps de notre simulation
N_t = 1000 # Temps maximal de notre simulation

### Définition des paramètres physiques

Posons d'abord les conditions limites de notre problème.

In [59]:
T_0 = 300  # Température initiale du corps étudié, en Kelvin (environ 27°C)
# Il est conseillé de garder 300 K : la densité et la capacité thermique d'un corps 
# sont thermosensibles, et plus loin on les prendra à 300 K.

T_1 = 323.15  # Température imposée au bord y=0, en Kelvin (50°C)
T_a = 273.15  # Température ambiante du fluide en x=Lx, en Kelvin (0°C)

Profitons-en pour donner quelques valeurs courantes pour des corps physiques usuels. 

En vérité, toutes les grandeurs qui peuvent servir à caractériser le comportement thermique d'un matériau ($\lambda$, $\rho$, $c_p$) sont elles-même thermosensibles. Pour se simplifier la vie, on les supposera constantes et égales à leur valeur à 300 K.

*(source : Transferts thermiques - J. Taine, F. Enguehard, E. Iacona - Éditions Dunod)*

In [60]:
class CorpsPhysique:
    def __init__(self, name, lambd, rho, cp):
        self.name = name  # Nom du corps physique
        self.lambd = lambd  # Conductivité thermique en W/(m·K)
        self.rho = rho  # Densité en kg/m³
        self.cp = cp  # Capacité thermique en J/(kg·K)
    
    def alpha(self):
        return self.lambd / (self.rho * self.cp)

air = CorpsPhysique("air", 0.0263, 1.1614, 1007)  # Air à 300 K
cuivre = CorpsPhysique("cuivre", 399, 8954, 383)  # Cuivre à 300 K

# Pour rajouter de nouveaux corps physiques, c'est ici.


Le calcul exact d'un coefficient de conducto-convection peut être délicat, et faire intervenir des nombres de Prandtl, Nusselt, la dilatation thermique et la gravité... Pour faire simple, nous imposerons un ordre de grandeur de h qu'on admettra comme cohérente pour de la convection naturelle laminaire sur une plaque plane (un mur).

In [61]:
# Si on veut changer le coefficient de convection extérieure, c'est ici.
h = 10 # en W/(m²·K)

# Si on veut changer le corps étudié, c'est ici.
corps = air

In [62]:
lambd, alpha = corps.lambd, corps.alpha()

# Calcul du temps caractéristique de conduction selon x
t_cond_x = L_x**2 / alpha
print(f"Temps caractéristique de conduction selon x: {t_cond_x:.2f}")

# Calcul du temps caractéristique de conduction selon y
t_cond_y = L_y**2 / alpha
print(f"Temps caractéristique de conduction selon y: {t_cond_y:.2f}")

print()

# Calcul du temps caractéristique de convection selon x
t_conv_x = t_cond_x / h
print(f"Temps caractéristique de convection selon x: {t_conv_x:.2f}")

# Calcul du temps caractéristique de convection selon y
t_conv_y = t_cond_y / h
print(f"Temps caractéristique de convection selon y: {t_conv_y:.2f}")

print()

# Calcul du nombre de Biot selon x
Bi_x = h * L_x / lambd
print(f"Nombre de Biot selon x: {Bi_x:.2f}")

# Calcul du nombre de Biot selon y
Bi_y = h * L_y / lambd
print(f"Nombre de Biot selon y: {Bi_y:.2f}")

print()

# Calcul du nombre de Fourier selon x
Fo_x = alpha * dt / dx**2
print(f"Nombre de Fourier selon x: {Fo_x:.5f}")

# Calcul du nombre de Fourier selon y
Fo_y = alpha * dt / dy**2
print(f"Nombre de Fourier selon y: {Fo_y:.5f}")


Temps caractéristique de conduction selon x: 4446881.37
Temps caractéristique de conduction selon y: 1111720.34

Temps caractéristique de convection selon x: 444688.14
Temps caractéristique de convection selon y: 111172.03

Nombre de Biot selon x: 3802.28
Nombre de Biot selon y: 1901.14

Nombre de Fourier selon x: 0.00225
Nombre de Fourier selon y: 0.00225


## Résolution analytique

L'énoncé indique que la solution analytique est donnée par la formule suivante :

$\displaystyle T(x,y) = T_a + \frac{2h}{\lambda} (T_1 - T_a) \sum_{n=1}^\infty \frac{\cos(\alpha_n x) \cosh(\alpha_n (L_y - y))}{[(\alpha_n^2 + \beta^2) L_x + \beta] \cos(\alpha_n L_x) \cosh(\alpha_n L_x)}$

où $\alpha_n$ désigne la nième solution positive de l'équation $\alpha \tan(L_x \alpha) = \beta = \displaystyle \frac{h}{\lambda}$.

Commençons donc par rédiger une fonction qui, à $n$ donné, permet d'obtenir $\alpha_n$ :

In [76]:
beta = h / lambd

def f(a):
    # Fonction dont on cherche les zéros - Ici la variable est nommée a
    # et non alpha, pour éviter toute confusion avec la diffusivité thermique 
    return a * np.tan(L_x * a) - beta



def n_premieres_solutions(n):
    # Liste pour stocker les solutions
    solutions = []

    # Estimation initiale de la solution - plus petit flottant positif
    a0 = np.finfo(float).eps

    while len(solutions) < n:
        solution = fsolve(f, a0)[0]

        # Si la solution est déjà dans la liste, l'ignorer
        if solution in solutions:
            continue

        # Ajout de la solution à la liste
        solutions.append(solution)

        # Mise à jour de l'estimation initiale pour la prochaine itération
        a0 += np.pi / L_x


    return solutions

print(n_premieres_solutions(10))



[2.220446049250313e-16, 4021.552755860297, -29.680260464569585, 41.30112361932847, 31.564723637850133, 26.539489339174658, 32.19287811715511, 19.315719148658978, 40.35889080550076, -18.059411699588175]


c:\Users\matte\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [75]:
# Definition of functions
def equation(x, L, h, l):
    return x * np.tan(L * x) - h / l

def find_first_n_solutions(L, h, l, n):
    solutions = []
    guess = 0.1
    while len(solutions) < n:
        solution = fsolve(equation, guess, args=(L, h, l))
        solution = solution[0]  # Convertir la solution en un nombre réel
        if not any(abs(sol - solution) < 1e-7 for sol in solutions):  # Vérifier si la solution est déjà dans la liste
            solutions.append(solution)
        guess += 0.1  # Augmenter la supposition initiale pour la prochaine solution
        # print(solutions)
    return solutions

print(find_first_n_solutions(L_x, h, lambd, 10))

def serie(Lx, Ly, h, l, x, y, n, liste_solutions):
    sum = 0
    for k in range(0, n+1):
        a = liste_solutions[k]
        x_k = (np.cos(a*y)*np.cosh(a*(Ly-x)))/(((a**2+(h/l)**2)*Lx+h/l)*np.cos(a*Lx)*np.cosh(a*Ly))
        sum += x_k
    # print("sum =", sum)
    return sum

def generate_analytical_profile(nx, ny, Lx, Ly, T1, Ta, h, k, n, liste_solutions):

    # Créer des grilles de points x et y
    x = np.linspace(0, Lx, nx)
    y = np.linspace(0, Ly, ny)

    # Initialiser le profil analytique
    T_analytical = np.zeros((nx, ny))

    # Calculer le profil analytique en utilisant une formule spécifique
    for i in range(ny):
        for j in range(nx):
            x = 1 - i/nx
            y = 1 - j/ny
            # Mettre en œuvre la formule analytique en fonction de x, y et d'autres paramètres
            sum = serie(Lx, Ly, h, k, x, y, n, liste_solutions)
            T_analytical[i, j] = (Ta + 2*(T1-Ta)*sum*h/k)

    # Tracer le profil analytique
    fig, ax = plt.subplots(figsize=(6, 6))
    im = ax.imshow(T_analytical, cmap='hot', interpolation='nearest')
    colorbar = fig.colorbar(im, ax=ax)
    colorbar.set_label('Température')
    # plt.imshow(T_analytical, cmap='hot', origin='lower', extent=[0, Lx, 0, Ly])
    # plt.colorbar(label='Température')
    # plt.xlabel('x')
    # plt.ylabel('y')
    plt.title('Profil de température analytique')
    # plt.gca().set_ylim(50, 0)
    plt.show()
    return T_analytical

[10.207492198353366, 30.308414812432122, 36.589960795607816, 6.75264844541422, -3.2818666268427266, 7.988684380277446, 1.7274216507058664, -6.107016070902613, -11.777875859751639, 1.4133449860960197]


c:\Users\matte\anaconda3\lib\site-packages\scipy\optimize\_minpack_py.py:178: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)
